https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, losses, util, InputExample, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [3]:
with open('dataframes\\all_wa_df.pickle', 'rb') as handle:
    all_wa_df = pickle.load(handle)

with open('dataframes\\all_stan_df.pickle', 'rb') as handle:
    all_stan_df = pickle.load(handle)

wa_content = all_wa_df.content
stan_content = all_stan_df.content



c:\Users\Nick\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


### Code to create training - dev - test dataset to finetune the model.

This is the first batch of articles evaluated on a 0 - 5 scale

In [ ]:
# df = pd.read_csv('ManualScoringLog.csv', delimiter=', ')
# df.top_3_standaard_idx = df.top_3_standaard_idx.apply(lambda x: x.strip("[]").replace("'","").split(","))
# df.Manual_Scores = df.Manual_Scores.apply(lambda x: x.strip("[]").replace("'","").split(","))
# WablieftIndexList = df['Wablieft_idx'].tolist()
# StandaardIndexList = df['top_3_standaard_idx'].tolist()
# SimScoreList = df['Manual_Scores'].tolist()

# TrainingExamples = []
# for i in range(0, len(WablieftIndexList)):
#     for j in range(0, 3):
#         texts =  [wa_content[i], stan_content[int(StandaardIndexList[i][j])]]
#         simScore = SimScoreList[i][j]
#         TrainingExamples.append(InputExample(texts=texts, label=float(simScore)/5))

In [8]:
# X_train, X_devtest = train_test_split(TrainingExamples, test_size=0.3)
# X_dev, X_test = train_test_split(X_devtest, test_size=0.5)

In [9]:
# # create evaluatordata
# testSentences1 = []
# testSentences2 = []
# testScores = []
# for testData in X_test:
#     testSentences1.append(testData.texts[0])
#     testSentences2.append(testData.texts[1])
#     testScores.append(testData.label)
    
# evaluator = EmbeddingSimilarityEvaluator(testSentences1, testSentences2, testScores)

In [2]:
## First evaluated wastan article pairs


with open('traindevtest\WaStan_FirstEval_train.pickle', 'rb') as handle:
    X_train= pickle.load(handle)

with open('traindevtest\WaStan_FirstEval_dev.pickle', 'rb') as handle:
    X_dev = pickle.load(handle)

with open('traindevtest\WaStan_FirstEval_test.pickle', 'rb') as handle:
    X_test = pickle.load(handle)

In [3]:
print(len(X_train))
print(len(X_test))

1140
245


#### robbertv2 mqa finetuned -- finetune on wastan

In [13]:
model_name = 'jegormeister/robbert-v2-dutch-base-mqa-finetuned'
train_batch_size = 6
num_epochs = 5
model_save_path = 'models\\robbert-v2-dutch-base-mqa-wastan-finetuned-final'

# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name, device="cuda")

In [14]:
train_dataloader = DataLoader(X_train, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(X_dev, name='wastan-dev')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [15]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

In [9]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(X_test, name='wastan-test')
test_evaluator(model, output_path=model_save_path)

0.3954667441374188

#### Robbert base v2

In [3]:
model_name = 'pdelobelle/robbert-v2-dutch-base'

In [11]:
train_batch_size = 6
num_epochs = 5
model_save_path = 'models\\robbert-v2-dutch-base-wastan-finetuned'

In [5]:
# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

In [6]:
train_dataloader = DataLoader(X_train, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(X_dev, name='wastan-dev')

In [7]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up

In [8]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

Iteration:   0%|          | 0/190 [00:00<?, ?it/s]

In [12]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(X_test, name='wastan-test')
test_evaluator(model, output_path=model_save_path)

0.4331896129515784